In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)






In [2]:
print(os.listdir("Temario\Modulo_2\TASMANIA\Data"))

['calendar.csv', 'listing.2.csv', 'listings.csv', 'neighbourhoods.csv', 'reviews.2.csv', 'reviews.csv']


In [ ]:
listings = pd.read_csv(r"", index_col= "id")
listings_details = pd.read_csv(r"", index_col= "id", low_memory=False)
calendar = pd.read_csv(r"", parse_dates=['date'], index_col=['listing_id'])
reviews_details = pd.read_csv(r"", parse_dates=['date']) 


In [ ]:
print(listings.shape)

(6230, 17)


In [5]:
listings.columns

Index(['name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood',
       'latitude', 'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm', 'license'],
      dtype='object')

El archivo listados_detalles contiene un total de 96 variables. No voy a utilizar todas estas, pero sí unir selectivamente una serie de variables que me parecieron útiles para esta EDA.

In [6]:
target_columns = ["property_type", "accommodates", "first_review", "review_scores_value", "review_scores_cleanliness", "review_scores_location", "review_scores_accuracy", "review_scores_communication", "review_scores_checkin", "review_scores_rating", "maximum_nights", "listing_url", "host_is_superhost", "host_about", "host_response_time", "host_response_rate", "price"]
listings = pd.merge(listings, listings_details[target_columns], on='id', how='left')
listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6230 entries, 35644 to 1236778914439262044
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            6230 non-null   object 
 1   host_id                         6230 non-null   int64  
 2   host_name                       6230 non-null   object 
 3   neighbourhood_group             0 non-null      float64
 4   neighbourhood                   6230 non-null   object 
 5   latitude                        6230 non-null   float64
 6   longitude                       6230 non-null   float64
 7   room_type                       6230 non-null   object 
 8   price_x                         5513 non-null   float64
 9   minimum_nights                  6230 non-null   int64  
 10  number_of_reviews               6230 non-null   int64  
 11  last_review                     5865 non-null   object 
 12  reviews_per_month   

In [7]:
listings.isnull().sum() / listings.shape[0] * 100   

name                                0.000000
host_id                             0.000000
host_name                           0.000000
neighbourhood_group               100.000000
neighbourhood                       0.000000
latitude                            0.000000
longitude                           0.000000
room_type                           0.000000
price_x                            11.508828
minimum_nights                      0.000000
number_of_reviews                   0.000000
last_review                         5.858748
reviews_per_month                   5.858748
calculated_host_listings_count      0.000000
availability_365                    0.000000
number_of_reviews_ltm               0.000000
license                             4.863563
property_type                       0.000000
accommodates                        0.000000
first_review                        5.858748
review_scores_value                 5.858748
review_scores_cleanliness           5.858748
review_sco

In [8]:
from sklearn.impute import KNNImputer

# Convertir la columna 'price_x' a numérica (eliminar símbolos de moneda y convertir a float)
listings['price_x'] = listings['price_x'].replace('[\$,]', '', regex=True).astype(float)

# Seleccionar las columnas que se utilizarán para la imputación
columns_for_imputation = ['price_x', 'accommodates', 'review_scores_rating']

# Crear una copia del dataframe solo con las columnas seleccionadas
listings_imputation = listings[columns_for_imputation]

# Crear el imputador KNN
imputer = KNNImputer(n_neighbors=5)

# Realizar la imputación
listings_imputed = imputer.fit_transform(listings_imputation)

# Reemplazar la columna 'price_x' en el dataframe original con los valores imputados
listings['price_x'] = listings_imputed[:, 0]

# Verificar los resultados
listings['price_x'].isnull().sum()

np.int64(0)

In [ ]:
# Check if the column 'neighbourhood_group' exists before dropping it
if 'neighbourhood_group' in listings.columns:
	listings = listings.drop(columns=['neighbourhood_group'])

listings['host_response_rate'] = pd.to_numeric(listings['host_response_rate'].str.strip('%'))
listings.head()



,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price_x,minimum_nights,number_of_reviews,...,review_scores_communication,review_scores_checkin,review_scores_rating,maximum_nights,listing_url,host_is_superhost,host_about,host_response_time,host_response_rate,price_y
id,,,,,,,,,,,,,,,,,,,,,
35644,Central Hobart Historic Cottage,153268,Yvette,Hobart,-42.87895,147.31526,Private room,214.2,1,255,...,4.96,4.96,4.82,7,https://www.airbnb.com/rooms/35644,t,I am an adjunct Senior Lecturer at the School ...,within an hour,100.0,NaN
84581,Kentisbury Country House Cradle Coast Tasmania,461471,Melissa,Kentish,-41.38892,146.23196,Entire home/apt,299.0,2,135,...,4.98,4.98,4.98,90,https://www.airbnb.com/rooms/84581,t,"Life revolves around family, food and operatin...",within an hour,100.0,$299.00
88873,Terrace- 5mins to central Hobart,485048,Russell And Ingrid,Hobart,-42.88219,147.32253,Entire home/apt,125.0,2,617,...,4.87,4.93,4.67,365,https://www.airbnb.com/rooms/88873,t,We're Russell and Ingrid.\r\nIngrid is a schoo...,within an hour,100.0,$125.00
137808,Bradman House CBD Launceston Invermay + Free WIFI,673625,Emma And Dale,Launceston,-41.42548,147.12855,Entire home/apt,239.0,1,412,...,4.68,4.82,4.53,1125,https://www.airbnb.com/rooms/137808,f,NaN,within an hour,100.0,$239.00
252996,Cosy Cabin @ Eagles Rise - Cradle Mountain 1 hour,1090010,Kit,Central Coast,-41.23440,146.13398,Entire home/apt,110.0,2,81,...,4.95,4.91,4.91,7,https://www.airbnb.com/rooms/252996,t,"From London originally, now living here in Tas...",within an hour,100.0,$110.00


In [10]:
listings.to_csv('data_Tasmania.csv', index=False)